In [ ]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276

In [1]:
#from scipy.io.wavfile import read
#import wave
import numpy as np
import os
import sox
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
import matplotlib.pyplot as plt
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import lfilter
import librosa
import math
import sys

## Functions

In [2]:
#Apply pre-emphasis (high-pass) filter
def apply_preEmph(x):
    x_filt = lfilter([1., -0.63], 1, x)
    return x_filt
        
#Obtain autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[int((result.size+1)/2):] #Note: other people use re.size/2:, but this does not work for me 
                                   # TODO: check consistency in other computers
#Get fundamental frequency (F0)
def get_F0(x,fs):
    #autocorrelation-based method to extract F0
    xcorr_arr = autocorr(x)
    
    #looking for F0 in the frequency interval 50-500Hz, but we search in time domain
    min_ms = round(fs/500)
    max_ms = round(fs/50)
    
    xcorr_slot = xcorr_arr[max_ms+1:2*max_ms+1]
    xcorr_slot = xcorr_slot[min_ms:max_ms]
    t0 = np.argmax(xcorr_slot)
    F0 = fs/(min_ms+t0-1)
    return F0

#Get formants
def get_formants(x, lp_order):
    #LP-based method
    
    #compute lp coefficients
    a = librosa.lpc(x, lp_ord)

    #get roots from lp coefficients
    rts = np.roots(a)
    rts = [r for r in rts if np.imag(r) >= 0]

    #get angles
    angz = np.arctan2(np.imag(rts), np.real(rts))

    #get formant frequencies
    formants = sorted(angz * (fs_targ / (2 * math.pi)))
    
    return formants

#Extract frequencies
def feature_extraction(x,fs,feats_df,lp_ord,ID,label):
#Extract features from signal x (identified as ID), and concatenate them to dataframe feats_df
#Features' reference: (see Appendix)
#[1]https://link.springer.com/article/10.1007/s10439-013-0741-6
#[2]https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?dsi_version=c5434db897ab74b192ca295a9eeca041&Expires=1585086202&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=c8k8DmG~KIxg0ToTO8rebm2MzHneCzJGkjSFRB7BYTEQ-MHXEr0ocHmISrldP3hFf9qmeiL11ezyefcNeRVeKIQ9PVjOl9pn7rXWcjA1o2voPn1VnDd8n7G2cT31apdj0LNMclhlXRPnCsGD66qDRqa3d-xaqqXhEqU73aw3ZgBgroO213MfJOqFhJxxXo2QEia0bSlDRTeX9KhSczFK-IFTPC6GwFL2L04por8pQRI3HF7E3f26O9zp9OhkwxSU9qfJah20WxZLA4PxREdv7JGoVBinR6T0mTcIaQi~B4IzYjSPSsTTADMNk5znVYIvSqgtMT~DY~qwlfq4SRdFjQ__
  
    #0)Wavelets #TODO
    
    #HAMMING WINDOW USED IN MFCC TODO: CHECK IF USING THAT ONE
    #DOUBT: if log-energy feature is included, should I also include the first mfcc coefficient (c0) ?
    #1)mfcc
    mfcc_feat = spe_feats.mfcc(x,fs)
    
    #TODO:consider also adding deltas and deltadeltas from the mfcc feat to capture temporal variations
          
    #2)zero-crossing rate
    zcr_feat = (((x[:-1] * x[1:]) < 0).sum())/len(x)
    
    #3)Formant frequencies
    #using LP-coeffcs-based method
    form= get_formants(x, lp_ord)
    
    #we keep just the first 4 formants
    formant_feats = form[0:4]
    
    #4)Log-energy
    logEnergy_feat = np.log10( ( (np.power(x,2)).sum()/len(x) ) + eps)    
    
    #5)Pitch (F0)
    F0_feat = get_F0(x,fs)
    
    #6)Kurtosis
    kurt_feat = kurtosis(x)
    
    #7)Bispectrum Score (BGS)
    #TODO
    
    #8)Non-Gaussianity Score (NGS)
    #TODO
    # Do kernel density estimation
    #p = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(seg_iarray)
    #q generate Gaussian distribution? using mean and standard dev of reference data, plus min and max values?
    
    #9) Adding skewness as measure of non-gaussianity (not in paper)
    skew_feat = skew(x)
    
    #DOUBT: 10) Shannon entropy GETTING -inf in all cases, WHY??? Don't include until fixed
    #entropy_feat = entropy(x)
    #Maybe compute directly to check

    feats_df = feats_df.append(pd.DataFrame({'Id': ID, 'mfcc': [mfcc_feat], 
                                       'kurtosis': kurt_feat, 'logEnergy': logEnergy_feat, 'zcr': zcr_feat,
                                       'formants': [formant_feats], 'F0': F0_feat, 'skew': skew_feat,
                                             'label': label}, index=[0]), ignore_index=True, sort=False)
    return feats_df


## Settings

In [3]:
#Initialize data frame of features:

feats = pd.DataFrame([])

#tiny constant value
eps = sys.float_info.epsilon

#Features' settings:

fs_targ = 16000 # set all audios to this sampling frequency
n_channels_targ = 1

#framing
winlen=0.025
winstep=0.01

#mfcc
mfcc_coeffcs= 12 #as paper (https://link.springer.com/article/10.1007/s10439-013-0741-6)

lp_ord = int(round(2 + fs_targ/1000)) #standard rule of thumb for LP oder

## Main

In [4]:
norm_skip = False #skip normalization step (because it has been done previously)

#s = read(audiofile)
FOLDER_PATH = 'data/YT_set/wavs/1/'
for file_name in os.listdir(FOLDER_PATH):
    
    fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
    #full path file name
    full_fname = FOLDER_PATH+file_name
    #print(full_fname)
    
    #TODO: put normalized wavs in other folder
    #name for normalization
    NORM_FOLDER_PATH = 'data/YT_set/wavs/norm/'
    norm_fname = NORM_FOLDER_PATH + os.path.splitext(file_name)[0] + '_NORM.wav'
    
    if norm_skip is False: 
        ## Normalization
        
        #level to same dB
        tfm = sox.Transformer()
        tfm.gain(gain_db=0.0, normalize=False, limiter=False, balance=None)
        #downsample to 16kHz and 1 channel
        tfm.convert(samplerate=fs_targ, n_channels=n_channels_targ, bitdepth=None) 
        #tfm.norm(db_level=0.0)
    
        # create the output normalized audio
        
        print(norm_fname)
        tfm.build(full_fname, norm_fname)
        tfm.effects_log
    
    # load normalized audio
    s = AudioSegment.from_wav(norm_fname)
    #sampling rate:
    info = mediainfo(norm_fname)
    fs = float(info['sample_rate'])
    
    #get ID of recording
    ID = fname_noExt.split('-')[-2] #for the current type of naming
    print(file_name)
    print(ID)
    
    #get label
    label = fname_noExt.split('-')[-1] #for the current type of naming
    print(label)
    
    ## Segmentation of cough streams (silence-based)
    #min_silence_len in ms, silence_thresh in dB
    s_segments = split_on_silence (s, min_silence_len = 600, silence_thresh = -30)
    ## TODO: set more accurate thresholds, or find other way to split (variance-based?)
    
    #convert s_segments to numpy array format
    AudioSegment2numpy_arr = lambda x: np.asarray(x.get_array_of_samples())
    s_segments_np = list(map(AudioSegment2numpy_arr, s_segments))
    
    print('High-pass filtering...')
    #pre-emphasis filtering to each segment
    preEmph_filtering = lambda x: apply_preEmph(x)
    s_segments_filt = list(map(preEmph_filtering, s_segments_np))
    
    #TODO
    #2) the segment is divided into X non-overlapping subsegments (X=3 for dry/wet cough paper,
    #X=12 for pneumonia paper)
    #TODO: window framing: I think maybe the segments need to be windowed with non-overlapping frames? 
    #(see frame settings at beggining of notebook)
    
    print('Computing features...')
    #Feature extraction for each segment
    
    #(lambda function doesn't work )
    #feat_extr_step = lambda x, fs, feats_df, lp_ord, ID: feature_extraction(x,fs,feats_df,lp_ord,ID)
    #feats = feat_extr_step(s_segments_filt,fs,feats,lp_ord,ID)
    for idx, seg_i in enumerate(s_segments_filt):
        print(idx)
        feats = feature_extraction(seg_i,fs,feats,lp_ord,ID,label)
    
       

output_file: data/YT_set/wavs/norm/Another Girl Coughing-iYxUHA-Pwsk-Dry_NORM.wav already exists and will be overwritten on build


data/YT_set/wavs/norm/Another Girl Coughing-iYxUHA-Pwsk-Dry_NORM.wav
Another Girl Coughing-iYxUHA-Pwsk-Dry.wav
Pwsk
Dry
High-pass filtering...
Computing features...
0
1
2
3
4


In [5]:
feats

,Id,mfcc,kurtosis,logEnergy,zcr,formants,F0,skew,label
0,Pwsk,"[[-36.04365338911715, 0.0, -3.2076317224819155...",10.261335,7.033567,0.266776,"[615.4712130184763, 1009.0686183582355, 2031.2...",213.333333,-0.035899,Dry
1,Pwsk,"[[9.60457382248319, -30.766157147256642, -13.5...",7.269807,7.445741,0.352220,"[795.7480969526919, 900.8716953508034, 2145.66...",152.380952,-0.003134,Dry
2,Pwsk,"[[9.681963295223953, -29.406913382520816, -7.9...",0.573200,6.289370,0.097091,"[551.2383040575314, 1045.736114981098, 1655.39...",163.265306,-0.180193,Dry
3,Pwsk,"[[9.888757595969555, -25.51711795819935, 0.038...",5.542743,7.493706,0.326300,"[715.756479741867, 1567.4161317150094, 1734.37...",326.530612,-0.019812,Dry
4,Pwsk,"[[10.639917255049568, -25.79301496979985, -2.0...",10.933372,7.087280,0.259046,"[558.7012952572038, 1242.9710489844333, 2103.9...",320.000000,-0.132976,Dry


### Pre-processing of features

In [ ]:
#TODO: Pre-proces the dataframe prior to use with a model (normalize if needed for the model, check there is no NaNs...)





## Model training

In [ ]:
#Cough sound
#Breathing rate
#Breathing rhytm (consistence smoothness)
#Cough rate
#Panic level
#Hoarseness